# PCA_Exp tutorial

This tutorial teaches how to use, and explains basic functionality of pca_exp package. The main task of pca_exp is to take a set of different experimental measurements, preprocess them, perform principal component analysis (PCA) and present the result in a user friendly format. Let's start from the quick explanation of PCA and why it is useful in analysis of experimental data.

## PCA 

